In [1]:
import pandas as pd
import os

In [2]:
tables_path = '../Other_Files/Таблицы'
tables = [filename for filename in os.listdir(tables_path) if os.path.isfile(f'{tables_path}/{filename}')]

In [6]:
tables_dfs = {}

for table in tables:
    df = pd.read_excel(f'{tables_path}/{table}')
    df = df[['CLASSID', 'NAME', 'ADDRESS', 'STATUS', 'REG_STATUS']] if 'REG_STATUS' in df.columns else df[['CLASSID', 'NAME', 'ADDRESS', 'STATUS']]

    tables_dfs[table.split('.xlsx')[0]] = df

---

In [8]:
def join_dfs(df_name: str) -> pd.DataFrame:
    statuses = {
        1: 'существующий',
        2: 'планируемый',
        3: 'планируемый к реконструкции',
        4: 'планируемый к ликвидации'
    }
    
    try:
        df = tables_dfs[df_name]

        values = pd.read_excel(f'{tables_path}/Values/{df_name}.xlsx')[1:].rename(columns={'Код объекта' : 'CLASSID'})

        joined = df.set_index('CLASSID').join(values[values.columns[:2]].set_index('CLASSID'))
        
        joined = joined.reset_index()
        joined['STATUS'] = joined['STATUS'].replace(statuses)


        # print(df)
        return(joined)


    except:     # Ловим ошибку, когда нет файлов
        return False


In [9]:
dfs_joined = {}

for key, df in tables_dfs.items():
    res = join_dfs(key)
    print(key, True if not isinstance(res, bool) else res)

    dfs_joined[key] = res


Cemetery True
Education True
Health True
Heritage True
HeritageArea True
Social True
Sport True


In [16]:
for key, df in dfs_joined.items():
    print(f"|{key}|         {'Все ок' if None not in df['Значение'].values else 'ЕСТЬ ПУСТЫЕ ЗНАЧЕНИЯ'}")
    print(dfs_joined[key][['STATUS', 'Значение']])


|Cemetery|         Все ок
          STATUS  Значение
0   существующий  Кладбище
1   существующий  Кладбище
2   существующий  Кладбище
3   существующий  Кладбище
4   существующий  Кладбище
5   существующий  Кладбище
6   существующий  Кладбище
7   существующий  Кладбище
8   существующий  Кладбище
9   существующий  Кладбище
10  существующий  Кладбище
11  существующий  Кладбище
12  существующий  Кладбище
13  существующий  Кладбище
14  существующий  Кладбище
15  существующий  Кладбище
16  существующий  Кладбище
17  существующий  Кладбище
18  существующий  Кладбище
19  существующий  Кладбище
|Education|         Все ок
           STATUS                                           Значение
0    существующий  Организация, реализующая программы профессиона...
1    существующий  Организация, реализующая программы профессиона...
2    существующий  Организация, реализующая программы профессиона...
3    существующий  Организация, реализующая программы профессиона...
4    существующий  Организация, реа

In [30]:
test_df = dfs_joined[list(dfs_joined.keys())[1]][['STATUS', 'Значение']].to_dict()

In [11]:
None in dfs_joined[list(dfs_joined.keys())[0]]['Значение'].copy().replace({'Кладбище' : None}).values

True

In [12]:
None in dfs_joined[list(dfs_joined.keys())[0]]['Значение'].copy().values

False

In [79]:
text_df = [f'№{counter} | Тип объекта:{test_df[list(test_df.keys())[1]][counter]} | Статус объекта:{test_df[list(test_df.keys())[0]][counter]}' for counter in range(len(test_df[list(test_df.keys())[0]]))][:20]

In [95]:
import requests as rq
model = 'llama3.2'
# prompt = 'Сколько всего объектов и какие уникальные статусы и типы объектов есть?'
prompt = 'How many objects are there in total and what unique statuses and types of objects are there? Translate your answer into Russian.'
# prompt = '1-посчитай, сколько объектов находятся в уникальных статусах;\n2-Посчитай, сколько объектов принадлежит к уникальным типам объектов;\n3-ВНИМАТЕЛЬНО СЛЕДИ ЗА СВОИМИ ПОДСЧЕТАМИ. НЕ ОШИБАЙСЯ. ОТВЕЧАЙ В ФОРМАТЕ СПЛОШНОГО ОТЧЕТА ТИПА СОЧИНЕНИЯ, А НЕ СПИСКОМ'

configs = {
    "model": model,
    "messages": [
        {
            "role": "user",
            "content": f'{prompt}\nОбщее число записей = {len(text_df)}\n Нумерованный список прилагаю ниже:\n' + '\n'.join(text_df)
        }
    ],


    'stream' : False}

proto = 'https'
url = 'zitrax-ollama-carbon.rshare.io/'



In [96]:
print(len(configs['messages'][0]['content']), configs['messages'][0]['content'])

2226 How many objects are there in total and what unique statuses and types of objects are there? Translate your answer into Russian.
Общее число записей = 20
 Нумерованный список прилагаю ниже:
№0 | Тип объекта:Организация, реализующая программы профессионального и высшего образования | Статус объекта:существующий
№1 | Тип объекта:Организация, реализующая программы профессионального и высшего образования | Статус объекта:существующий
№2 | Тип объекта:Организация, реализующая программы профессионального и высшего образования | Статус объекта:существующий
№3 | Тип объекта:Организация, реализующая программы профессионального и высшего образования | Статус объекта:существующий
№4 | Тип объекта:Организация, реализующая программы профессионального и высшего образования | Статус объекта:существующий
№5 | Тип объекта:Общеобразовательная организация | Статус объекта:существующий
№6 | Тип объекта:Организация дополнительного образования | Статус объекта:существующий
№7 | Тип объекта:Организация 

In [97]:
response = rq.post(f'{proto}://{url}/api/chat', json=configs, verify=False)


j:\SelfWrited\University\7_semester_project\.venv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'zitrax-ollama-carbon.rshare.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [98]:
response
print(response.json()['message']['content'])

Всего существует 20 записей.

Объекты можно разделить на следующие категории:

1. Организации, реализующие программы профессионального и высшего образования:
 * Эта категория включает в себя 10 записей (№№ 0, 1, 2, 3, 4, 9, 10, 14, 15, 19).
 * Все эти объекты являются существующими.
2. Общеобразовательные организации:
 * Эта категория включает в себя 4 записи (№№ 5, 6, 11, 16).
 * Все эти объекты являются существующими.
3. Организации дополнительного образования:
 * Эта категория включает в себя 8 записей (№№ 7, 12, 13, 17, 18).
 * Все эти объекты являются существующими.
4. Дошкольные образовательные организации:
 * Эта категория включает в себя 6 записей (№ 8, № 11, № 17, № 18, № 19).
 * Все эти объекты являются существующими.

Вместе все эти категории представляют собой общее количество существующих объектов, обозначающих различные типы организаций и educational institutions.
